In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
%matplotlib inline
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
import warnings
from sklearn.preprocessing import OrdinalEncoder
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import chi2_contingency
#pd.set_option('display.max_rows', None)
%run functions.ipynb

In [2]:
#print(display())

# Reading Data

In [3]:
data = read_csv('regression_data_clean.csv')

In [4]:
data.head(5)

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price
0,7129300520,10/13/14,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650,221900
1,6414100192,12/9/14,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639,538000
2,5631500400,2/25/15,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062,180000
3,2487200875,12/9/14,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000,604000
4,1954400510,2/18/15,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503,510000


In [5]:
#data.drop_duplicates()


# sorting out the categorical columns in our Data

In [6]:
data['bedrooms'] = data['bedrooms'].astype(str)

In [7]:
data['bedrooms'].dtype

dtype('O')

In [8]:
data['waterfront'] = data['waterfront'].astype(str)

In [9]:
data['view'] = data['view'].astype(str)

In [10]:
data['floors'] = data['floors'].astype(str)

In [11]:
data['condition'] = data['condition'].astype(str)

In [12]:
data['grade'] = data['grade'].astype(str)

In [13]:
data['bathrooms'] = data['bathrooms'].astype(str)

In [14]:
data['zipcode'] = data['zipcode'].astype(str)

# removing 'id' and 'date' columns as 'id' has all the unique values.

In [15]:
data = col_drop(data,['id'])

In [16]:
data = col_drop(data,['date'])

# exploring the column bathroom variables

In [17]:
data['bathrooms'].value_counts()

2.5     5377
1.0     3851
1.75    3048
2.25    2047
2.0     1930
1.5     1445
2.75    1185
3.0      753
3.5      731
3.25     589
3.75     155
4.0      136
4.5      100
4.25      79
0.75      71
4.75      23
5.0       21
5.25      13
5.5       10
1.25       9
6.0        6
0.5        4
5.75       4
6.5        2
6.25       2
8.0        2
6.75       2
7.75       1
7.5        1
Name: bathrooms, dtype: int64

# Removing outliers from the data using the variable 'sqft_lot' 

In [18]:
iqr = np.percentile(data['sqft_lot'],75) - np.percentile(data['sqft_lot'],25)
upper_limit = np.percentile(data['sqft_lot'],75) + 3*iqr
lower_limit = np.percentile(data['sqft_lot'],25) - 3*iqr
data = data[(data['sqft_lot']>lower_limit) & (data['sqft_lot']<upper_limit)]
data.shape

(19828, 19)

# splitting the data into categorical and numerical variables

In [19]:
X_num,X_cat = data_split(data)

# Distribution plots 

In [20]:
#for val in X_num.columns:
  #  distribution_plot(X_num,val)

In [21]:
#for val in X_num.columns:
#    sns.distplot(X_num[val])
#    plt.show()

# applying log transformation on 'sqft_above', 'sqft_basement', 'sqft_living15'

In [22]:
my_list = ['yr_built', 'yr_renovated','lat','long','price','sqft_living', 'sqft_lot15','sqft_lot']
my_list1 = my_difference(X_num.columns,my_list)
print(my_list1)

['sqft_above', 'sqft_basement', 'sqft_living15']


In [23]:
for val in my_list1:
    X_num[val] = X_num[val].apply(lambda x: np.log(x) if(x>0) else x)
    #sns.displot(X_num[val])


# yeo johnson transformation 

In [24]:
#power = PowerTransformer(method='yeo-johnson', standardize=True)
#data_trans = power.fit_transform(X_num['lat'].values.reshape(-1,1))
#data_trans = [item[0] for item in data_trans]
#X_num['lat'] = data_trans
#data_trans1 = power.fit_transform(X_num['yr_built'].values.reshape(-1,1))
#data_trans1 = [item[0] for item in data_trans1]
#X_num['yr_built'] = data_trans1

#  boxplot helpful for analyzing outliers

In [25]:
#for val in X_num.columns:
#    sns.boxplot(X_num[val])
#    plt.show()

# finding the colinearity between the numerical variables

In [26]:
#my_heatMap(X_num,X_num.columns)

In [27]:
#col = ['sqft_lot15','sqft_above','sqft_living15']
#X_num = col_drop(X_num,col)

# Variance inflation implementation on numerical variables

In [28]:
#X_num = VIF(X_num)
#X_num.head(5)

In [29]:
#for val in X_cat.columns:
 #   print(X_cat[val].unique())

# Adjusting yr_renovated variable as a categorical variable

In [30]:
def renovation(x):
    if x == 0:
        x = 'not renovated'
        return x
    else:
          x = 'renovated'
    return x

In [31]:
X_num['yr_renovated']= X_num['yr_renovated'].apply(renovation)

In [32]:
X_cat['yr_renovated']= X_num['yr_renovated']

In [33]:
X_cat.head(5)

,bedrooms,bathrooms,floors,waterfront,view,condition,grade,zipcode,yr_renovated
0,3,1.0,1.0,0,0,3,7,98178,not renovated
1,3,2.25,2.0,0,0,3,7,98125,renovated
2,2,1.0,1.0,0,0,3,6,98028,not renovated
3,4,3.0,1.0,0,0,5,7,98136,not renovated
4,3,2.0,1.0,0,0,3,8,98074,not renovated


In [34]:
X_num = X_num.drop(['yr_renovated'], axis = 1)

In [35]:
X_num.columns

Index(['sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement', 'yr_built',
       'lat', 'long', 'sqft_living15', 'sqft_lot15', 'price'],
      dtype='object')

In [36]:
X_num.head(5)


,sqft_living,sqft_lot,sqft_above,sqft_basement,yr_built,lat,long,sqft_living15,sqft_lot15,price
0,1180,5650,7.073270,0.000000,1955,47.5112,-122.257,7.200425,5650,221900
1,2570,7242,7.682482,5.991465,1951,47.7210,-122.319,7.432484,7639,538000
2,770,10000,6.646391,0.000000,1933,47.7379,-122.233,7.908387,8062,180000
3,1960,5000,6.956545,6.813445,1965,47.5208,-122.393,7.215240,5000,604000
4,1680,8080,7.426549,0.000000,1987,47.6168,-122.045,7.495542,7503,510000


In [37]:
X_cat.head(5)

,bedrooms,bathrooms,floors,waterfront,view,condition,grade,zipcode,yr_renovated
0,3,1.0,1.0,0,0,3,7,98178,not renovated
1,3,2.25,2.0,0,0,3,7,98125,renovated
2,2,1.0,1.0,0,0,3,6,98028,not renovated
3,4,3.0,1.0,0,0,5,7,98136,not renovated
4,3,2.0,1.0,0,0,3,8,98074,not renovated


# Applying chi_square contigency test on categorical variables

In [38]:
#lis = []
#for i in range((X_cat.shape[1]) - 1):
#    lis.append([])
#    for j in range(i+1, X_cat.shape[1]):
#        data_crosstab = pd.crosstab(X_cat.iloc[:,i], X_cat.iloc[:,j] , margins = False)
#        chi2_contingency(data_crosstab, correction=False)
#        [statistic, p_value, dof, arr] = chi2_contingency(data_crosstab)
#        lis[i].append(p_value)
#print(lis)
#X_cat = X_cat[['bathrooms','bathrooms', 'floors','waterfront','view', 'condition', 'grade', 'zipcode', 'yr_renovated']]

# Applying ordinalEncoder label to categorical variables

In [39]:
#x_cat = pd.get_dummies(X_cat,drop_first = True)
enc = OrdinalEncoder()
enc.fit(X_cat)
x_cat = enc.transform(X_cat)
x_cat1 = pd.DataFrame(x_cat)

# Applying oneHotEncoder label to categorical variables

In [40]:
#enc = OneHotEncoder()
#enc.fit(X_cat)
#x_cat = enc.transform(X_cat)
#x_cat1 = pd.DataFrame(x_cat)

# Storing the price into target variable y

In [41]:
y = X_num['price']
X_num = X_num.drop(['price'],axis = 1)

# concatanating numerical and categorical variables after storing the (price) into target variables
  

In [42]:
df = pd.concat([X_num.reset_index(drop=True), x_cat1.reset_index(drop=True)], axis=1)
df.head(6)

,sqft_living,sqft_lot,sqft_above,sqft_basement,yr_built,lat,long,sqft_living15,sqft_lot15,0,1,2,3,4,5,6,7,8
0,1180,5650,7.073270,0.000000,1955,47.5112,-122.257,7.200425,5650,4.0,2.0,0.0,0.0,0.0,2.0,8.0,66.0,0.0
1,2570,7242,7.682482,5.991465,1951,47.7210,-122.319,7.432484,7639,4.0,7.0,2.0,0.0,0.0,2.0,8.0,55.0,1.0
2,770,10000,6.646391,0.000000,1933,47.7379,-122.233,7.908387,8062,3.0,2.0,0.0,0.0,0.0,2.0,7.0,16.0,0.0
3,1960,5000,6.956545,6.813445,1965,47.5208,-122.393,7.215240,5000,6.0,10.0,0.0,0.0,0.0,4.0,8.0,58.0,0.0
4,1680,8080,7.426549,0.000000,1987,47.6168,-122.045,7.495542,7503,4.0,6.0,0.0,0.0,0.0,2.0,9.0,37.0,0.0
5,1715,6819,7.447168,0.000000,1995,47.3097,-122.327,7.713338,6819,4.0,7.0,2.0,0.0,0.0,2.0,8.0,2.0,0.0


# Normalizer implementation

In [43]:
#transformer = Normalizer().fit(df)
#x_normalized = transformer.transform(df) 

# Standard Scaler implementation

In [44]:
scaler = preprocessing.StandardScaler().fit(df)
x_scaled = scaler.transform(df)
x_scaled = pd.DataFrame(x_scaled)
#for i in range(x_scaled.shape[1]):
 #   sns.distplot(x_scaled[i])
  #  plt.show()

# Data splitting

In [45]:
X_train, X_test, y_train, y_test = train_test_split(x_scaled, y, test_size = 0.2, random_state=100)

# Models Implementation

In [46]:
model1 = linear_model.LinearRegression()
model2 = KNeighborsRegressor(7)
model3 = RandomForestRegressor(random_state=0)
linear_models = [model1,model2,model3]
my_list,my_list1 = models_automation(linear_models, X_train, y_train, X_test, y_test)

LinearRegression(): Test -> 0.7043200097605926
KNeighborsRegressor(n_neighbors=7): Test -> 0.8133956817540035
RandomForestRegressor(random_state=0): Test -> 0.8890341108713773


# Showing scores of three models we have applied 

In [53]:
data1 = my_scores(my_list)
data1.head()

,0,1,2
Models,Linear Regression,KNneighborsRegressor,RandomForestRegressor
R_square,0.704320,0.813396,0.889034
MAE,127787.482906,87101.608854,66997.259393
MSE,38135304405.983276,24067277850.016750,14311817168.196821
RMSE,195282.626995,155136.320216,119632.007290


# The actul and predicted prices of all models

In [54]:
predicted_val= pd.DataFrame({'Actual_price': y_test,'LinearRegression': my_list1[0][0],
                             'KNeighborsRegressor': my_list1[1][0],
                             'RandomForestRegressor':
                             my_list1[2][0]})
predicted_val[['LinearRegression','KNeighborsRegressor','RandomForestRegressor']] = predicted_val[
    ['LinearRegression','KNeighborsRegressor','RandomForestRegressor']
    ].astype('int64')
predicted_val.head()

,Actual_price,LinearRegression,KNeighborsRegressor,RandomForestRegressor
18763,235000,371508,317564,308410
7487,841000,839792,844571,809053
9857,290000,382125,306771,317742
15638,419950,442215,460857,422217
18314,2920000,1698888,2187142,2500200


# The average of actual and predicted prices of all variables

In [55]:
average_dataframe = pd.DataFrame({'average actual_price': [529871] ,
                                   'average linear_regression': [528879] ,
                                   'average kN_regression': [527034],
                                   'average randomForest_regression': [530321]})
average_dataframe.head()

,average actual_price,average linear_regression,average kN_regression,average randomForest_regression
0,529871,528879,527034,530321
